In [1]:
!pip install crewai crewai_tools langchain-huggingface chromadb streamlit pyngrok boto3

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.2/191.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.7/463.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

# Writing the Backend file

In [2]:
%%writefile sqlite_backend.py
import sqlite3
import os

# Define the path for the SQLite database
DB_PATH = os.path.join(os.getcwd(), "investment_analysis.db")

# Initialize or connect to an SQLite database and create the table if it doesn't exist
def init_db():
    """
    Initialize the SQLite database and create the 'analysis_data' table if it doesn't exist.
    """
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS analysis_data (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                topic TEXT NOT NULL,
                parameters TEXT NOT NULL,
                content TEXT NOT NULL
            )
        ''')
        conn.commit()
        print("Database initialized and table 'analysis_data' is ready.")
    except sqlite3.Error as e:
        print(f"Error initializing database: {e}")
    finally:
        conn.close()

# Check if the 'analysis_data' table exists
def check_table_exists():
    """
    Check if the 'analysis_data' table exists in the database.
    :return: True if the table exists, False otherwise.
    """
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute('''
            SELECT name FROM sqlite_master WHERE type='table' AND name='analysis_data';
        ''')
        result = cursor.fetchone()
        if result:
            print("Table 'analysis_data' exists.")
            return True
        else:
            print("Table 'analysis_data' does not exist.")
            return False
    except sqlite3.Error as e:
        print(f"Error checking table existence: {e}")
        return False
    finally:
        conn.close()

# Save data to SQLite
def save_to_sqlite(topic, parameters, content):
    """
    Save content related to stock analysis or investment advice to SQLite.
    :param topic: Topic for the analysis (e.g., stock symbol or investment strategy)
    :param parameters: String containing user parameters (e.g., initial capital, risk tolerance)
    :param content: Analysis or advice content to save
    """
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO analysis_data (topic, parameters, content)
            VALUES (?, ?, ?)
        ''', (topic, parameters, content))
        conn.commit()
        print("Content saved to SQLite database.")
    except sqlite3.Error as e:
        print(f"Error saving content to SQLite: {e}")
    finally:
        conn.close()

# Retrieve data from SQLite
def retrieve_from_sqlite(topic, parameters):
    """
    Retrieve saved content from SQLite based on topic and parameters.
    :param topic: Topic for the analysis (e.g., stock symbol or investment strategy)
    :param parameters: String containing user parameters (e.g., initial capital, risk tolerance)
    :return: The saved content or a message if not found
    """
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute('''
            SELECT content FROM analysis_data
            WHERE topic = ? AND parameters = ?
        ''', (topic, parameters))
        result = cursor.fetchone()
        if result:
            print("Content retrieved successfully.")
            return result[0]  # Return the content
        else:
            print("No content found for the given topic and parameters.")
            return "No content found for the given topic and parameters."
    except sqlite3.Error as e:
        print(f"Error retrieving content from SQLite: {e}")
        return "An error occurred while retrieving the content."
    finally:
        conn.close()

# Delete data from SQLite based on topic and parameters
def delete_from_sqlite(topic, parameters):
    """
    Delete saved content from SQLite based on topic and parameters.
    :param topic: Topic for the analysis (e.g., stock symbol or investment strategy)
    :param parameters: String containing user parameters (e.g., initial capital, risk tolerance)
    :return: Message indicating whether deletion was successful or not.
    """
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute('''
            DELETE FROM analysis_data
            WHERE topic = ? AND parameters = ?
        ''', (topic, parameters))
        conn.commit()
        if cursor.rowcount > 0:
            print("Content deleted successfully.")
            return "Content deleted successfully."
        else:
            print("No content found to delete for the given topic and parameters.")
            return "No content found to delete for the given topic and parameters."
    except sqlite3.Error as e:
        print(f"Error deleting content from SQLite: {e}")
        return "An error occurred while deleting the content."
    finally:
        conn.close()

# Retrieve all data from the database
def retrieve_all_data():
    """
    Retrieve all saved records from the 'analysis_data' table.
    :return: List of all records, or an empty list if no data is found
    """
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM analysis_data')
        records = cursor.fetchall()
        print("All records retrieved successfully.")
        return records
    except sqlite3.Error as e:
        print(f"Error retrieving all records: {e}")
        return []
    finally:
        conn.close()

# Update existing data in SQLite based on topic and parameters
def update_sqlite(topic, parameters, new_content):
    """
    Update existing content in the SQLite database based on topic and parameters.
    :param topic: Topic for the analysis (e.g., stock symbol or investment strategy)
    :param parameters: String containing user parameters (e.g., initial capital, risk tolerance)
    :param new_content: New content to update in the database
    :return: Message indicating whether update was successful or not.
    """
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        cursor.execute('''
            UPDATE analysis_data
            SET content = ?
            WHERE topic = ? AND parameters = ?
        ''', (new_content, topic, parameters))
        conn.commit()
        if cursor.rowcount > 0:
            print("Content updated successfully.")
            return "Content updated successfully."
        else:
            print("No matching content found to update.")
            return "No matching content found to update."
    except sqlite3.Error as e:
        print(f"Error updating content in SQLite: {e}")
        return "An error occurred while updating the content."
    finally:
        conn.close()

Writing sqlite_backend.py


# Writing into the Main File

In [4]:
%%writefile app.py
import streamlit as st
import os
import yfinance as yf
import requests
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv
from sqlite_backend import init_db, save_to_sqlite, retrieve_from_sqlite, check_table_exists

# Load environment variables
load_dotenv()

# Set OpenAI API key and NewsAPI key
os.environ["OPENAI_API_KEY"] = "<OPEN_AI_API_KEY>"
NEWSAPI_KEY = "<NEWSAPI_API_KEY" # Make sure to add your NewsAPI key in the .env file

# Ensure the SQLite database is initialized
init_db()

# Verify that the table exists
if not check_table_exists():
    st.error("Table 'analysis_data' could not be created. Please check your database setup.")
else:
    print("Table 'analysis_data' is confirmed to exist.")

# Streamlit configuration
st.set_page_config(page_title="Stock Market Analysis & Investment (India)", page_icon="💹")
st.title("Stock Market Analysis & Investment (India)")

# Initialize session state for storing analysis content
if "analysis_content" not in st.session_state:
    st.session_state.analysis_content = ""

# Input fields for trading inputs
stock_symbol = st.text_input("Enter Stock Symbol (e.g., RELIANCE)")
initial_capital = st.number_input("Enter Initial Capital in ₹ (e.g., 100000)", min_value=0)
risk_tolerance = st.selectbox("Select Risk Tolerance", ["Low", "Moderate", "High"])
trading_strategy = st.text_input("Enter Trading Strategy (e.g., Value Investing, Momentum)")
investment_horizon = st.selectbox("Investment Horizon", ["Short-term", "Long-term"])
portfolio_diversification = st.number_input("Portfolio Diversification Percentage", min_value=0, max_value=100)
period = st.selectbox("Select Data Period", ["1mo", "3mo", "6mo", "1y", "5y", "10y"])

# Display the initial capital and other inputs using the rupee symbol in output (if applicable)
st.write(f"Initial Capital: ₹{initial_capital}")

# Define a function to fetch stock data with technical indicators using yfinance
def fetch_stock_data(symbol, period="1y"):
    stock = yf.Ticker(symbol)
    data = stock.history(period=period)

    # Technical Indicators
    data['MA50'] = data['Close'].rolling(window=50).mean()
    data['MA200'] = data['Close'].rolling(window=200).mean()
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))
    data['Upper_BB'] = data['Close'].rolling(window=20).mean() + (data['Close'].rolling(window=20).std() * 2)
    data['Lower_BB'] = data['Close'].rolling(window=20).mean() - (data['Close'].rolling(window=20).std() * 2)

    return data

# Fetch fundamental data
def fetch_fundamental_data(symbol):
    stock = yf.Ticker(symbol)
    info = stock.info
    return {
        "Market Cap": info.get("marketCap"),
        "P/E Ratio": info.get("trailingPE"),
        "EPS": info.get("trailingEps"),
        "Debt/Equity Ratio": info.get("debtToEquity")
    }

# Fetch real-time news using NewsAPI
def fetch_realtime_news(stock_symbol):
    url = f"https://newsapi.ai/api/v1/news?apikey={NEWSAPI_KEY}&q={stock_symbol}&language=en"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            news_data = response.json()
            news_articles = [
                {"title": article["title"], "description": article["description"]}
                for article in news_data["articles"][:5]  # Limit to the top 5 articles
            ]
            return news_articles
        else:
            print("Error fetching news:", response.status_code, response.text)
            return []
    except Exception as e:
        print("Error fetching news:", e)
        return []

# Display stock data with indicators
if stock_symbol:
    stock_data = fetch_stock_data(stock_symbol, period)
    st.write(f"Stock Data with Indicators for {stock_symbol}")
    st.line_chart(stock_data[['Close', 'MA50', 'MA200', 'Upper_BB', 'Lower_BB']])
    st.line_chart(stock_data['RSI'])

# Display fundamental data and economic indicators
fundamentals = fetch_fundamental_data(stock_symbol) if stock_symbol else {}
st.write("Fundamental Analysis")
st.json(fundamentals)

st.write("Real-Time News")
news_articles = fetch_realtime_news(stock_symbol)
for article in news_articles:
    st.write(f"{article['title']}\n{article['description']}")

# Define Stock Analyst and Investment Advisor Agents
stock_analyst = Agent(
    role='Stock Analyst',
    goal=f'Analyze market trends, price patterns, and performance indicators for {stock_symbol}.',
    verbose=True,
    memory=True,
    backstory="Expert in stock market analysis, providing key insights on price patterns, trends, and potential risks.",
    allow_delegation=True
)

# Pass the latest news as part of the goal for investment advice
investment_advisor = Agent(
    role='Investment Advisor',
    goal=(
        f"Provide investment advice for {stock_symbol}, based on an initial capital of ₹{initial_capital}, "
        f"a {risk_tolerance} risk tolerance, and a {trading_strategy} strategy. Consider a {investment_horizon} horizon, "
        f"{portfolio_diversification}% diversification, technical indicators, fundamentals, and recent news updates. "
        f"The latest news includes the following: {news_articles}."
    ),
    verbose=True,
    memory=True,
    backstory="Knowledgeable in finance and investment, offers insights into optimizing capital allocation and managing risks.",
    allow_delegation=False
)

# Define Tasks for Analysis and Investment Advice
analysis_task = Task(
    description=f"Analyze stock {stock_symbol} using technical indicators and fundamentals over {period} with a {investment_horizon} perspective.",
    expected_output="Summary of stock trends, key indicators, and risk factors relevant to {stock_symbol}.",
    agent=stock_analyst
)

advice_task = Task(
    description=f"Recommend an investment strategy for {stock_symbol} with an initial capital of ₹{initial_capital}, "
                f"considering {risk_tolerance} risk, a {investment_horizon} horizon, and {portfolio_diversification}% diversification.",
    expected_output="Investment strategy with risk management techniques and capital allocation tips, incorporating recent news updates.",
    agent=investment_advisor,
    async_execution=False,
    output_file='investment-advice.md'
)

# Button to Generate Analysis and Investment Advice
if st.button("Generate Analysis & Advice"):
    if stock_symbol and initial_capital > 0:
        crew = Crew(
            agents=[stock_analyst, investment_advisor],
            tasks=[analysis_task, advice_task],
            process=Process.sequential,
            memory=True,
            cache=True,
            max_rpm=100,
            share_crew=True
        )

        result = crew.kickoff(inputs={
            'stock_symbol': stock_symbol,
            'initial_capital': initial_capital,
            'risk_tolerance': risk_tolerance,
            'trading_strategy': trading_strategy,
            'investment_horizon': investment_horizon,
            'portfolio_diversification': portfolio_diversification
        })

        if os.path.exists("investment-advice.md"):
            with open("investment-advice.md", "r") as file:
                st.session_state.analysis_content = file.read()
                st.markdown(st.session_state.analysis_content)
        else:
            st.error("Failed to generate analysis and advice. Please try again.")
    else:
        st.error("Please enter all required trading inputs.")

# Button to Save Analysis and Advice
if st.button("Save"):
    if st.session_state.analysis_content:
        parameters = f"₹{initial_capital}-{risk_tolerance}-{trading_strategy}"
        save_to_sqlite(stock_symbol, parameters, st.session_state.analysis_content)
        st.success("Analysis and advice saved successfully!")
    else:
        st.error("Please generate the analysis and advice first before saving.")

# Button to Retrieve Analysis and Advice
if st.button("Retrieve Previous Analysis"):
    if stock_symbol and initial_capital > 0:
        parameters = f"₹{initial_capital}-{risk_tolerance}-{trading_strategy}"
        previous_content = retrieve_from_sqlite(stock_symbol, parameters)
        if previous_content:
            st.markdown(previous_content)
        else:
            st.error("No previous analysis found for these parameters.")
    else:
        st.error("Please enter the required inputs to retrieve previous analysis.")

Writing app.py


# Hosting using NGROK and Streamlit

In [5]:
# Install necessary packages
# !pip install streamlit boto3 pyngrok

# Import ngrok and start the tunnel
from pyngrok import ngrok

# Set up ngrok authentication with your auth token
ngrok.set_auth_token("<NGROK_AUTH_TOKEN>")

# Start the Streamlit app in the background
get_ipython().system_raw('streamlit run app.py &')  # Starts Streamlit on default port 8501

# Start ngrok tunnel for HTTP protocol on port 8501
public_url = ngrok.connect(addr="8501", proto="http")
print(f"Access your Streamlit app here: {public_url}")


Access your Streamlit app here: NgrokTunnel: "https://5e35-34-70-150-182.ngrok-free.app" -> "http://localhost:8501"


Note: Please dont click on the link that you see. It is a saved output of a previous execution .